In [1]:
import operator
from functools import partial

In [2]:
with open('inputs/day-05.txt', 'r') as f:
    parsed = f.read()[:-1]  # drop eof newline

code = list(map(int, parsed.split(',')))
# print(parsed)
print(code[:100])

[3, 225, 1, 225, 6, 6, 1100, 1, 238, 225, 104, 0, 1102, 46, 47, 225, 2, 122, 130, 224, 101, -1998, 224, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 6, 224, 1, 224, 223, 223, 1102, 61, 51, 225, 102, 32, 92, 224, 101, -800, 224, 224, 4, 224, 1002, 223, 8, 223, 1001, 224, 1, 224, 1, 223, 224, 223, 1101, 61, 64, 225, 1001, 118, 25, 224, 101, -106, 224, 224, 4, 224, 1002, 223, 8, 223, 101, 1, 224, 224, 1, 224, 223, 223, 1102, 33, 25, 225, 1102, 73, 67, 224, 101, -4891]


## part 1

In [3]:
def pad(n):
    return f"{n:05d}"


def resolve(tape, val, mode):
    return tape[val] if mode == 0 else val


def _math(func, tape, pointer, params, modes, *args):
    a = resolve(tape, params[0], modes[0])
    b = resolve(tape, params[1], modes[1])
    return int(func(a, b)), params[2], pointer, True


def _inp(tape, pointer, params, modes, inp):
    return inp, params[0], pointer, True


def _out(tape, pointer, params, modes, *args):
    val = resolve(tape, params[0], modes[0])
    print("output:", val)
    return None, None, pointer, True


def _exec_pt1(compute_func, num_params, store, tape, pointer, inp=None):

    def _modes(n):
        d = dict(enumerate(reversed(pad(n)[:3])))
        return {k: int(v) for k, v in d.items()}
    
    modes = _modes(tape[pointer])
    params = tape[pointer + 1:pointer + num_params + 1]

    val, store_at, pointer, _ = compute_func(tape, pointer, params, modes, inp)

    if store:
        tape = list(tape)  # deepcopy
        tape[store_at] = val
        
    return tape, (pointer + num_params + 1)


def process_pt1(tape, inp, pointer=0):
    instructions = {
        1: partial(_exec_pt1, partial(_math, operator.add), 3, True),
        2: partial(_exec_pt1, partial(_math, operator.mul), 3, True),
        3: partial(_exec_pt1, _inp, 1, True),
        4: partial(_exec_pt1, _out, 1, False),
    }
    opcode = int(str(tape[pointer])[-2:])
    _tape, _pointer = instructions[opcode](tape, pointer, inp)

    if _tape[_pointer] == 99:
        print("halt")
        return _tape
    return process_pt1(_tape, inp, _pointer)


process_pt1(code, 1, 0);

output: 0
output: 0
output: 0
output: 0
output: 0
output: 0
output: 0
output: 0
output: 0
output: 12896948
halt


## part 2

In [4]:
def _jump(func, tape, pointer, params, modes, *args):
    cond = resolve(tape, params[0], modes[0])
    if func(cond):
        pointer = resolve(tape, params[1], modes[1])
        return None, None, pointer, False
    return None, None, pointer, True


def _exec_pt2(compute_func, num_params, store, tape, pointer, inp=None):

    def _modes(n):
        d = dict(enumerate(reversed(pad(n)[:3])))
        return {k: int(v) for k, v in d.items()}
    
    modes = _modes(tape[pointer])
    params = tape[pointer + 1:pointer + num_params + 1]

    val, store_at, pointer, forward = compute_func(tape, pointer, params, modes, inp)

    if store:
        tape = list(tape)  # deepcopy
        tape[store_at] = val
        forward = (pointer != store_at)
        
    return tape, (pointer + num_params + 1) if forward else pointer


def process_pt2(tape, inp, pointer=0):
    instructions = {
        1: partial(_exec_pt2, partial(_math, operator.add), 3, True),
        2: partial(_exec_pt2, partial(_math, operator.mul), 3, True),
        3: partial(_exec_pt2, _inp, 1, True),
        4: partial(_exec_pt2, _out, 1, False),
        5: partial(_exec_pt2, partial(_jump, bool), 2, False),
        6: partial(_exec_pt2, partial(_jump, lambda x: not bool(x)), 2, False),
        7: partial(_exec_pt2, partial(_math, operator.lt), 3, True),
        8: partial(_exec_pt2, partial(_math, operator.eq), 3, True),
    }
    opcode = int(str(tape[pointer])[-2:])
    _tape, _pointer = instructions[opcode](tape, pointer, inp)

    if _tape[_pointer] == 99:
        print("halt")
        return _tape
    return process_pt2(_tape, inp, _pointer)


process_pt2(code, 5, 0);

output: 7704130
halt
